In [83]:
import kfp
import kfp.components as comp
import kfp.dsl as dsl
import kfp.compiler as cpl
from kfp.components import InputPath, InputTextFile, OutputPath, OutputTextFile
from typing import Dict # Biblioteca nativa do python
import os 

In [49]:
# Parâmetros Opcionais
BASE_IMAGE = 'tensorflow/tensorflow:1.11.0-py3'

# O Dataset já vem pré-processado e normalizado
DATASET = 'https://raw.githubusercontent.com/ambientelivre/samples_kubeflow/main/datasets/bolsa.csv'

PIPELINE_NAME = 'Pipeline Previsao Dolar'
PIPELINE_DESCRIPTION = 'Dados Bolsa X Dolar'

PIPELINES_PATH = 'pipelines'
OUTPUT_PIPELINE_PATH = os.path.join(PIPELINES_PATH,'pipe_bolsa.yaml')

if not os.path.exists(PIPELINES_PATH):
    os.makedirs(PIPELINES_PATH)

EXPERIMENT_NAME = 'TESTES-AMBIENTE-BOLSA-X-DOLAR'
RUN_NAME = 'PIPELINE-RUN-BOLSA-X-DOLAR'

# MODEL_NAME = 'bolsa_pipeline_model_' + str(int(time.time())) 
# OUTPUT_MODEL_PATH = 'models/'

# MODEL_VERSION = 'bolsa_pipeline_model_v1' + str(int(time.time())) 

In [50]:
def treina(feat:str, label:str, dataset:str, output_model_path:OutputPath(str)) :
    """
    A função retorna o caminho até o binário do arquivo que contém o modelo treinado.
    Isso fica implícito quando declaramos OutputPath(str).
    """

    import pickle
    import pandas as pd
    from sklearn.linear_model import LinearRegression
    
    # Recebe o dataset e treina o algoritmo
    df = pd.read_csv(dataset)
    reglin = LinearRegression()
    reglin.fit(df[[feat]], df[label])
    
    # Registra o binário do modelo treinado no caminho até o arquivo "output_model_path" no minIO
    with open(output_model_path, 'wb') as f:
        pickle.dump(reglin, f)
    

In [51]:
comp_treina = comp.create_component_from_func(treina, output_component_file='treina_component.yaml',
                                              base_image=BASE_IMAGE)

In [52]:
def treina_2(feat:str, label:str, dataset:str, output_model_path:OutputPath(str)) :
    """
    A função retorna o caminho até o binário do arquivo que contém o modelo treinado.
    Isso fica implícito quando declaramos OutputPath(str).
    """

    import pickle
    import pandas as pd
    from sklearn.tree import DecisionTreeRegressor

    # Recebe o dataset e treina o algoritmo
    df = pd.read_csv(dataset)
    dtree = DecisionTreeRegressor(max_depth=5,  #1 para perder, 5 pra ganhar
                                  random_state=0)
    dtree.fit(df[[feat]], df[label])

    # Registra o binário do modelo treinado no caminho até o arquivo "output_model_path" no minIO
    with open(output_model_path, 'wb') as f:
        pickle.dump(dtree, f)

In [53]:
comp_treina_2 = comp.create_component_from_func(treina_2, output_component_file='treina_2_component.yaml',
                                              base_image=BASE_IMAGE)

In [54]:
def valida(feat:str, label:str, dataset:str, model_path:InputPath()) -> Dict[str,float]:
    
    import pickle
    import pandas as pd
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    
    # Importando o dataset
    df = pd.read_csv(dataset)
    y_true = df[label]
    
    # Importando o modelo serializado como um objeto python
    with open(model_path, 'rb') as f:
        trained_model = pickle.load(f)
    
    y_pred = trained_model.predict(df[[feat]])
    
    # Calcula as métricas de desempenho do modelo para o problema de regressão
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    
    valida_metrics = {
        'r2' : r2,
        'mse' : mse, # comentar oq são
        'mae' : mae
    }
    
    # Retorna um dicionário com o cálculo das métricas

    return valida_metrics

In [55]:
comp_valida = comp.create_component_from_func(valida, output_component_file='valida_component.yaml'
                                              ,base_image=BASE_IMAGE)

In [56]:
def compara(metric:str, metrics1:Dict[str,float], metrics2:Dict[str,float]) -> int:
    
    val1 = float(metrics1[metric])
    val2 = float(metrics2[metric])
    
    if (metric == 'mse' or metric == 'mae'):
        val1 = -val1
        val2 = -val2
    
    if (val2 > val1) :
        return 2
    elif (val2 < val1) :
        return 1
    else: 
        return 0
    
# compara('r2', {'r2':18.6}, {'r2':9.6})

In [57]:
comp_compara = comp.create_component_from_func(compara, 
                                                 output_component_file='compara_component.yaml',
                                                 base_image='python:alpine3.6')

In [58]:
def previsao(feat:str, label:str, model_path:InputPath(), valor:float) -> float:

    import pandas as pd
    import pickle
    
    future = pd.DataFrame({feat:[valor]})
    with open(model_path, 'rb') as f:
        trained_model = pickle.load(f)

    # Retorna a previsão do valor futuro do dólar quando a bolsa atinge ${valor} pontos
        
    return trained_model.predict(future)[0]
    

In [59]:
comp_previsao = comp.create_component_from_func(previsao, output_component_file='previsao_component.yaml'
                                                ,base_image=BASE_IMAGE)

In [60]:
@dsl.pipeline(
    name=PIPELINE_NAME,
    description=PIPELINE_DESCRIPTION
)
def pipe(feat, label, valor, dataset, metric):

    # Treinamento do algoritmo 1 (mais simples)
    task_treina_1 = comp_treina(feat, label, dataset)

    # Treinamento do algoritmo 2 (mais complexo)
    task_treina_2 = comp_treina_2(feat, label, dataset)

    
    # Validação do modelo 1
    task_valida_1 = comp_valida(feat, label, dataset, task_treina_1.output)
    
    # Validação do modelo 2
    task_valida_2 = comp_valida(feat, label, dataset, task_treina_2.output)
    
    
    # Compara a métrica desejada na validação dos modelos
    task_compara = comp_compara(metric, task_valida_1.output, task_valida_2.output)
    

    # Caso o desempenho do modelo 1 na métrica desejada for igual ou superior ao do modelo 2:
    # Essa forma de se fazer é interessante pois dsl.Condition() só permite usar uma única comparação
    # A comparação pode ser com int, float, string, ...
    with dsl.Condition(task_compara.output < 2): 

        # Previsão final usando o modelo 1
        task_previsao = comp_previsao(feat, label, task_treina_1.output, valor)
    
    # Caso o desempenho do modelo 2 na métrica desejada for superior ao do modelo 1
    with dsl.Condition(task_compara.output==2):

        # Previsão final usando o modelo 2
        task_previsao = comp_previsao(feat, label, task_treina_2.output, valor)
    


In [61]:
# Instanciando um objeto-cliente para integração com o Kubeflow
client = kfp.Client(host='http://localhost:8080/pipeline')

In [62]:
args = {
'feat': 'bolsa',
'label': 'usd',
'valor': '110',
'dataset': DATASET,
'metric' : 'r2'
}

# print(client.list_experiments())

In [63]:
# A execução do pipeline pode ser feita através do método create_run_from_pipeline_func()
# que carrega as componentes "comp_*" já instanciadas no código e também o pipeline
client.create_run_from_pipeline_func(pipe, arguments=args, experiment_name=EXPERIMENT_NAME, 
                                     run_name=RUN_NAME) 

RunPipelineResult(run_id=0ea3785c-ea2b-4930-8c05-95798de07b07)

In [64]:
# Outra forma de executarmos o pipeline é compilando ele gerando um .yaml 
# Desde que 
# que pode ser usado para criar e executar um pipeline
cpl.Compiler().compile(
    pipeline_func=pipe,
    package_path=OUTPUT_PIPELINE_PATH
)    

In [ ]:
client.create_run_from_pipeline_package(
    pipeline_file=OUTPUT_PIPELINE_PATH,
    arguments=args
)